<a href="https://colab.research.google.com/github/nimaone/adaptive_teacher/blob/main/MIC_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb  5 17:47:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.13.1+cu116 True
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 85.8 MB/s eta 0:00:01tcmalloc: large alloc 1837596672 bytes == 0x354e000 @  0x7fa5f947b680 0x7fa5f949c824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 92.8 MB/s eta 0:00:01tcmalloc: large alloc 2296995840 bytes == 0x70dc6000 @  0x7fa5f947b680 0x7fa5f949bda2 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a
tcmalloc: la

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
!pip install ninja==1.10.2.3 yacs==0.1.8 cython==0.29.28 matplotlib==3.5.1 tqdm==4.63.0 opencv-python==4.5.5.64


In [ ]:
!pip install timm==0.6.11 kornia==0.5.8 einops==0.4.1


In [47]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!python setup.py build_ext install

In [ ]:
%cd {HOME}
!git clone https://github.com/mcordts/cityscapesScripts.git
%cd cityscapesScripts/
!python setup.py build_ext install

In [48]:
%cd {HOME}
!git clone https://github.com/lhoyer/MIC
%cd MIC/det
# the following will install the lib with
# symbolic links, so that you can modify
# the files if you want and won't need to
# re-build it
!python setup.py build develop

/content
Cloning into 'MIC'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 455 (delta 63), reused 444 (delta 56), pack-reused 0
Receiving objects: 100% (455/455), 4.31 MiB | 9.24 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/MIC/det
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.8
creating build/lib.linux-x86_64-3.8/maskrcnn_benchmark
copying maskrcnn_benchmark/__init__.py -> build/lib.linux-x86_64-3.8/maskrcnn_benchmark
creating build/lib.linux-x86_64-3.8/maskrcnn_benchmark/modeling
copying maskrcnn_benchmark/modeling/utils.py -> build/lib.linux-x86_64-3.8/maskrcnn_benchmark/modeling
copying maskrcnn_benchmark/modeling/make_layers.py -> build/lib.linux-x86_64-3.8/maskrcnn_benchmark/modeling
copying maskrcnn_benchmark/modeling/balanced_positive_negative_sampler.py -> build/lib.linux-x86_64-3.8/maskrcnn_benchmark/modeling
copying mas

In [8]:
!ls

build		 INSTALL.md	     maskrcnn_benchmark.egg-info  tests
configs		 LICENSE	     README.md			  tools
environment.yml  maskrcnn_benchmark  setup.py


In [ ]:
%cd {HOME}

# copy required file to cityscapesScripts toolkit
!cp tool/cityscapes/instances2dict_with_polygons.py $INSTALL_DIR/cityscapesScripts/evaluation

# recompile cityscapesScripts
%cd cityscapesScripts/
!python setup.py build_ext install

!pip install h5py==3.6.0 scipy==1.8.0

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
%cd /content/

/content


In [ ]:
!gdown 1-1Iow33KqdaNQcrK4v9Zk568IZVYZ1UV
!gdown 1-0WQV7BerZb2ieF0LYzoFlO9NFjy4gy7
!gdown 1-1rhR_4ynP162kL1UrmWQex8tyj6Ki3F


In [ ]:
!wget https://deepblue.lib.umich.edu/data/downloads/ks65hc58r

In [ ]:
# !wget http://droplab-files.engin.umich.edu/repro_10k_annotations.tgz

In [ ]:
# !wget http://droplab-files.engin.umich.edu/repro_image_sets.tgz

In [9]:
!mkdir /content/datasets

In [ ]:
!tar -zxvf /content/repro_10k_images.tgz -C /content/datasets
!tar -zxvf /content/repro_10k_annotations.tgz -C /content/datasets
!tar -zxvf /content/repro_image_sets.tgz -C /content/datasets

In [12]:
!mv /content/datasets/VOC2012 /content/datasets/Sim10k


In [ ]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=kiaei&password=UtP8!pbXp8FsQs2&submit=Login' https://www.cityscapes-dataset.com/login/

In [ ]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [ ]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

In [16]:
!unzip -q /content/leftImg8bit_trainvaltest.zip -d /content/datasets

In [17]:
!unzip -q /content/gtFine_trainvaltest.zip -d /content/datasets/cityscapes

In [18]:
!mv /content/datasets/leftImg8bit /content/datasets/cityscapes

In [24]:
!rm -r  /content/leftImg8bit_trainvaltest.zip

In [ ]:
!gdown 1a2oHjcEcwXP8oUF95qiwrqzACb2YlUhn -O  /content/datasets/visdrone
!gdown 1bxK5zgLn0_L8x276eKkuYA_FzwCIjb59 -O  /content/datasets/visdrone
!gdown 1PFdW_VFSCfZ_sTSZAGjQdifF_Xd5mf0V -O  /content/datasets/visdrone

In [ ]:
# !unzip -q /content/datasets/visdrone/VisDrone2019-DET-train.zip -d /content/datasets/visdrone/train
# !unzip -q /content/datasets/visdrone/VisDrone2019-DET-val.zip -d /content/datasets/visdrone/val
# !unzip -q /content/datasets/visdrone/VisDrone2019-DET-test-challenge -d /content/datasets/visdrone/test

In [58]:
!mkdir /content/datasets/visdrone

In [59]:
!unzip -q /content/drive/MyDrive/UDA/VisDrone2019-DET-train.zip -d /content/datasets/visdrone
!unzip -q /content/drive/MyDrive/UDA/VisDrone2019-DET-val.zip -d /content/datasets/visdrone
!unzip -q /content/drive/MyDrive/UDA/VisDrone2019-DET-test-dev.zip -d /content/datasets/visdrone/test

In [60]:
import os
print(len(os.listdir('/content/datasets/visdrone/train/VisDrone2019-DET-train/images')))
print(len(os.listdir('/content/datasets/visdrone/train/VisDrone2019-DET-train/annotations')))
print(len(os.listdir('/content/datasets/visdrone/val/VisDrone2019-DET-val/images')))
print(len(os.listdir('/content/datasets/visdrone/val/VisDrone2019-DET-val/annotations')))

6471
6471
548
548


In [63]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

In [77]:
from pathlib import Path
import os
path= "/content/datasets/visdrone"  # dataset root dir
train= "VisDrone2019-DET-train/images"  # train images (relative to 'path')  6471 images
val= "VisDrone2019-DET-val/images"  # val images (relative to 'path')  548 images
test= "VisDrone2019-DET-test-dev/images"  # test images (optional)  1610 images

# Classes
names={
  0: "pedestrian",
  1: "people",
  2: "bicycle",
  3: "car",
  4: "van",
  5: "truck",
  6: "tricycle",
  7: "awning-tricycle",
  8: "bus",
  9: "motor"}


# Download script/URL (optional) ---------------------------------------------------------------------------------------
def visdrone2yolo(dir):
    from PIL import Image
    from tqdm import tqdm
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh
    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(os.sep + 'annotations' + os.sep, os.sep + 'labels' + os.sep), 'w') as fl:
                    fl.writelines(lines)  # write label.txt
  # Download
dir = Path(path)  # dataset root dir
  # urls = ['https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-train.zip',
  #         'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-val.zip',
  #         'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-dev.zip',
  #         'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-challenge.zip']
  # download(urls, dir=dir, curl=True, threads=4)
  # Convert
for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
     visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels

Converting /content/datasets/visdrone/VisDrone2019-DET-train: 6471it [00:39, 163.29it/s]
Converting /content/datasets/visdrone/VisDrone2019-DET-val: 548it [00:04, 122.49it/s]
Converting /content/datasets/visdrone/VisDrone2019-DET-test-dev: 0it [00:00, ?it/s]


In [68]:
!ls

cityscapesScripts		       instances2dict_with_polygons.py
convert_cityscapes_to_caronly_coco.py  leftImg8bit_trainvaltest.zip
convert_sim10k_to_coco.py	       MIC
cookies.txt			       __pycache__
datasets			       repro_10k_annotations.tgz
drive				       repro_10k_images.tgz
gtFine_trainvaltest.zip		       repro_image_sets.tgz
index.html			       sample_data


In [ ]:
!pip install fiftyone

In [2]:
import fiftyone as fo
from fiftyone import ViewField as F


In [3]:

imgs_path_train = "/content/datasets/visdrone/VisDrone2019-DET-train/images"
labels_path_train = "/content/datasets/visdrone/VisDrone2019-DET-train/labels"

imgs_path_val = "/content/datasets/visdrone/VisDrone2019-DET-val/images"
labels_path_val = "/content/datasets/visdrone/VisDrone2019-DET-val/labels"

# name = 'visdrone'
# The dataset or view to export
dataset_train = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv4Dataset,
    data_path=imgs_path_train,
    labels_path=labels_path_train,
)
# session = fo.launch_app(dataset)
dataset_val = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv4Dataset,
    data_path=imgs_path_val,
    labels_path=labels_path_val,
)

 100% |███████████████| 6471/6471 [2.8m elapsed, 0s remaining, 22.6 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 6471/6471 [2.8m elapsed, 0s remaining, 22.6 samples/s]      


 100% |█████████████████| 548/548 [19.0s elapsed, 0s remaining, 29.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 548/548 [19.0s elapsed, 0s remaining, 29.2 samples/s]      


In [4]:

# ANIMALS = [
#     "Sedan", "Truck",
# ]

# Replace all animal detection's labels with "animal"
# mapping = {k: "car" for k in ANIMALS}

# name_s = {k:k for k,v in names.items()}

names={
  '0': "pedestrian",
  '1': "people",
  '2': "bicycle",
  '3': "car",
  '4': "van",
  '5': "truck",
  '6': "tricycle",
  '7': "awning-tricycle",
  '8': "bus",
  '9': "motor"}

car_view_train = dataset_train.map_labels("ground_truth", names)
car_view_val = dataset_val.map_labels("ground_truth", names)

In [5]:
session = fo.launch_app(car_view_val)


In [6]:
from fiftyone import ViewField as F

car_view_train = car_view_train.filter_labels(
    "ground_truth", (F("label") == "car") 
)
car_view_val = car_view_val.filter_labels(
    "ground_truth", (F("label") == "car") 
)

In [7]:
fo.pprint(car_view_train.stats(include_media=True))
fo.pprint(car_view_val.stats(include_media=True))

Computing metadata...


INFO:fiftyone.core.metadata:Computing metadata...


 100% |███████████████| 6133/6133 [26.9s elapsed, 0s remaining, 284.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 6133/6133 [26.9s elapsed, 0s remaining, 284.0 samples/s]      


{
    'samples_count': 6133,
    'samples_bytes': 23361933,
    'samples_size': '22.3MB',
    'media_bytes': 1481450812,
    'media_size': '1.4GB',
    'total_bytes': 1504812745,
    'total_size': '1.4GB',
}
Computing metadata...


INFO:fiftyone.core.metadata:Computing metadata...


 100% |█████████████████| 515/515 [2.1s elapsed, 0s remaining, 276.1 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 515/515 [2.1s elapsed, 0s remaining, 276.1 samples/s]      


{
    'samples_count': 515,
    'samples_bytes': 2248960,
    'samples_size': '2.1MB',
    'media_bytes': 78288320,
    'media_size': '74.7MB',
    'total_bytes': 80537280,
    'total_size': '76.8MB',
}


In [ ]:
session = fo.launch_app(car_view_val)


In [8]:
# export_dir = "/content/dataset/Sim10k"
labels_path_train  = "/content/datasets/visdrone/VisDrone2019-DET-train/train_only_car.json"
labels_path_val  = "/content/datasets/visdrone/VisDrone2019-DET-val/val_only_car.json"

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
label_field = "ground_truth"
dataset_type = fo.types.COCODetectionDataset  # for example

car_view_train.export(
    labels_path=labels_path_train,
    dataset_type=dataset_type,
    label_field=label_field,
    export_media=False
)
car_view_val.export(
    labels_path=labels_path_val,
    dataset_type=dataset_type,
    label_field=label_field,
    export_media=False
)


 100% |███████████████| 6133/6133 [38.7s elapsed, 0s remaining, 102.9 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 6133/6133 [38.7s elapsed, 0s remaining, 102.9 samples/s]      


 100% |█████████████████| 515/515 [3.6s elapsed, 0s remaining, 165.9 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 515/515 [3.6s elapsed, 0s remaining, 165.9 samples/s]      


In [11]:
import fiftyone as fo
from fiftyone import ViewField as F


In [70]:

imgs_path = "/content/datasets/Sim10k/JPEGImages"
labels_path = "/content/datasets/Sim10k/Annotations"


# name = 'visdrone'
# The dataset or view to export
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv4Dataset,
    data_path=imgs_path,
    labels_path=labels_path,
)


 100% |███████████████| 6471/6471 [2.8m elapsed, 0s remaining, 27.2 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 6471/6471 [2.8m elapsed, 0s remaining, 27.2 samples/s]      


In [ ]:
session = fo.launch_app(dataset)


In [23]:
car_view = dataset.filter_labels(
    "ground_truth", (F("label") == "car") 
)

In [ ]:
session = fo.launch_app(view=car_view)


In [ ]:
# export_dir = "/content/datasets/Sim10k"
labels_path  = "/content/datasets/Sim10k/train_only_car.json"

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
label_field = "ground_truth"
dataset_type = fo.types.COCODetectionDataset  # for example

car_view.export(
    labels_path=labels_path,
    dataset_type=dataset_type,
    label_field=label_field,
    export_media=False
)

In [12]:
!cp /content/datasets/cityscapes/caronly_filtered_gtFine_test.json /content/drive/MyDrive/AI/dataset/cityscapes
!cp /content/datasets/cityscapes/caronly_filtered_gtFine_train.json /content/drive/MyDrive/AI/dataset/cityscapes
!cp /content/datasets/cityscapes/caronly_filtered_gtFine_val.json /content/drive/MyDrive/AI/dataset/cityscapes

In [13]:
!cp /content/datasets/Sim10k/car_instances.json /content/drive/MyDrive/AI/dataset/sim10k
!cp /content/datasets/Sim10k/train_only_car.json /content/drive/MyDrive/AI/dataset/sim10k

In [14]:
!cp /content/datasets/visdrone/VisDrone2019-DET-train/train_only_car.json /content/drive/MyDrive/AI/dataset/visdrone
!cp /content/datasets/visdrone/VisDrone2019-DET-val/val_only_car.json /content/drive/MyDrive/AI/dataset/visdrone

In [50]:
%cd /content/

/content


In [ ]:
!python convert_cityscapes_to_caronly_coco.py --dataset cityscapes_car_only\
  --outdir '/content/datasets/cityscapes'\
  --datadir '/content/datasets/cityscapes/'

In [ ]:
!python convert_sim10k_to_coco.py --dataset sim10k\
  --outdir '/content/datasets/Sim10k/'\
  --datadir '/content/datasets/Sim10k/Annotations'

In [35]:
import json

with open('/content/datasets/Sim10k/train_only_car.json') as user_file:
  file_contents = user_file.read()
  
parsed_json = json.loads(file_contents)
print(parsed_json.keys())
print(len(parsed_json["images"]))
print(len(parsed_json["annotations"]))

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
9975
57776


In [36]:
import json

with open('/content/datasets/Sim10k/car_instances.json') as user_file:
  file_contents = user_file.read()
  
parsed_json = json.loads(file_contents)
print(parsed_json.keys())
print(len(parsed_json["images"]))
print(len(parsed_json["annotations"]))

dict_keys(['images', 'type', 'annotations', 'categories'])
10000
57776


In [41]:
!unzip -q /content/mic.zip -d /content/sample_data

In [29]:
!mv  /content/datasets/cityscapes/leftImg8bit/train/*/*.png /content/datasets/cityscapes/leftImg8bit/train/
!mv  /content/datasets/cityscapes/leftImg8bit/val/*/*.png /content/datasets/cityscapes/leftImg8bit/val/
!mv  /content/datasets/cityscapes/leftImg8bit/test/*/*.png /content/datasets/cityscapes/leftImg8bit/test/


In [30]:
%cd /content/MIC/det

/content/MIC/det


In [37]:
!python tools/train_net.py --config-file /content/MIC/det/configs/fff.yaml

2023-02-04 16:08:26,097 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-04 16:08:26,098 maskrcnn_benchmark INFO: Namespace(config_file='/content/MIC/det/configs/fff.yaml', deterministic=False, distributed=False, local_rank=0, opts=[], seed=1, skip_test=False)
2023-02-04 16:08:26,098 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2023-02-04 16:08:27,434 maskrcnn_benchmark INFO: 
PyTorch version: 1.12.0+cu113
Is debug build: False
CUDA used to build PyTorch: 11.3
ROCM used to build PyTorch: N/A

OS: Ubuntu 20.04.5 LTS (x86_64)
GCC version: (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Clang version: 10.0.0-4ubuntu1 
CMake version: version 3.22.6
Libc version: glibc-2.31

Python version: 3.8.10 (default, Nov 14 2022, 12:59:47)  [GCC 9.4.0] (64-bit runtime)
Python platform: Linux-5.10.147+-x86_64-with-glibc2.29
Is CUDA available: True
CUDA runtime version: 11.2.152
GPU models and configuration: GPU 0: Tesla T4
Nvidia driver version: 510.47.03
cuDNN version: Probably one of the f

In [14]:
!python tools/train_net.py --config-file /content/sample_data/content/MIC/det/configs/da_faster_rcnn/fff.yaml


2023-02-03 13:28:47,742 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-03 13:28:47,742 maskrcnn_benchmark INFO: Namespace(config_file='/content/sample_data/content/MIC/det/configs/da_faster_rcnn/fff.yaml', deterministic=False, distributed=False, local_rank=0, opts=[], seed=1, skip_test=False)
2023-02-03 13:28:47,742 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2023-02-03 13:28:49,324 maskrcnn_benchmark INFO: 
PyTorch version: 1.12.0+cu113
Is debug build: False
CUDA used to build PyTorch: 11.3
ROCM used to build PyTorch: N/A

OS: Ubuntu 20.04.5 LTS (x86_64)
GCC version: (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Clang version: 10.0.0-4ubuntu1 
CMake version: version 3.22.6
Libc version: glibc-2.31

Python version: 3.8.10 (default, Nov 14 2022, 12:59:47)  [GCC 9.4.0] (64-bit runtime)
Python platform: Linux-5.10.147+-x86_64-with-glibc2.29
Is CUDA available: True
CUDA runtime version: 11.2.152
GPU models and configuration: GPU 0: Tesla T4
Nvidia driver version: 510.47.03
c

In [38]:
%cd /content/MIC/det

/content/MIC/det


In [40]:
!python tools/test_net.py --config-file "/content/MIC/det/configs/fff.yaml" MODEL.WEIGHT /content/drive/MyDrive/AI/mic/model_0006000.pth


2023-02-04 17:32:29,860 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-04 17:32:29,860 maskrcnn_benchmark INFO: DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  SOURCE_TRAIN: ('sim10k_cocostyle',)
  TARGET_TRAIN: ('cityscapes_only_car_train_cocostyle',)
  TEST: ('cityscapes_only_car_val_cocostyle',)
  TRAIN: ()
INPUT:
  MAX_SIZE_TEST: 1600
  MAX_SIZE_TRAIN: 1600
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN
    FREEZE_CONV_BODY_AT: 2
    OUT_CHANNELS: 256
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DA_HEADS:
    COS_WEIGHT: 0.1
    DA_IMG_GRL_WEIGHT: 0.01
    DA_INS_GRL_WEIGHT: 0.1
  DEVICE: cuda
  DOMAIN_ADAPTATION_ON: True
  FPN:
    USE_GN: False
    USE_RELU: False
  GROUP_NORM:
    DIM_PER_GP: -1
    EPSILON: 1e-05
    NUM_GROUPS: 32
  KEYPOINT_ON: False
  MASKING_AUGMENTATION: True
  MASKING_BLOC

In [42]:
!python tools/test_net.py --config-file "/content/MIC/det/configs/fff.yaml" MODEL.WEIGHT /content/drive/MyDrive/AI/mic/model_0005000.pth


2023-02-04 17:36:19,618 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-04 17:36:19,618 maskrcnn_benchmark INFO: DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  SOURCE_TRAIN: ('sim10k_cocostyle',)
  TARGET_TRAIN: ('cityscapes_only_car_train_cocostyle',)
  TEST: ('cityscapes_only_car_val_cocostyle',)
  TRAIN: ()
INPUT:
  MAX_SIZE_TEST: 1600
  MAX_SIZE_TRAIN: 1600
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN
    FREEZE_CONV_BODY_AT: 2
    OUT_CHANNELS: 256
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DA_HEADS:
    COS_WEIGHT: 0.1
    DA_IMG_GRL_WEIGHT: 0.01
    DA_INS_GRL_WEIGHT: 0.1
  DEVICE: cuda
  DOMAIN_ADAPTATION_ON: True
  FPN:
    USE_GN: False
    USE_RELU: False
  GROUP_NORM:
    DIM_PER_GP: -1
    EPSILON: 1e-05
    NUM_GROUPS: 32
  KEYPOINT_ON: False
  MASKING_AUGMENTATION: True
  MASKING_BLOC

In [43]:
!python tools/test_net.py --config-file "/content/MIC/det/configs/fff.yaml" MODEL.WEIGHT /content/drive/MyDrive/AI/mic/model_0004000.pth


2023-02-04 17:39:14,860 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-04 17:39:14,861 maskrcnn_benchmark INFO: DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  SOURCE_TRAIN: ('sim10k_cocostyle',)
  TARGET_TRAIN: ('cityscapes_only_car_train_cocostyle',)
  TEST: ('cityscapes_only_car_val_cocostyle',)
  TRAIN: ()
INPUT:
  MAX_SIZE_TEST: 1600
  MAX_SIZE_TRAIN: 1600
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN
    FREEZE_CONV_BODY_AT: 2
    OUT_CHANNELS: 256
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DA_HEADS:
    COS_WEIGHT: 0.1
    DA_IMG_GRL_WEIGHT: 0.01
    DA_INS_GRL_WEIGHT: 0.1
  DEVICE: cuda
  DOMAIN_ADAPTATION_ON: True
  FPN:
    USE_GN: False
    USE_RELU: False
  GROUP_NORM:
    DIM_PER_GP: -1
    EPSILON: 1e-05
    NUM_GROUPS: 32
  KEYPOINT_ON: False
  MASKING_AUGMENTATION: True
  MASKING_BLOC

In [18]:
!python tools/test_net.py --config-file "/content/fff.yaml" MODEL.WEIGHT /content/drive/MyDrive/AI/mic/model_0003000.pth


2023-02-03 14:50:59,612 maskrcnn_benchmark INFO: Using 1 GPUs
2023-02-03 14:50:59,612 maskrcnn_benchmark INFO: DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  SOURCE_TRAIN: ('sim10k_cocostyle',)
  TARGET_TRAIN: ('cityscapes_only_car_train_cocostyle',)
  TEST: ('cityscapes_only_car_val_cocostyle',)
  TRAIN: ()
INPUT:
  MAX_SIZE_TEST: 1600
  MAX_SIZE_TRAIN: 1600
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN
    FREEZE_CONV_BODY_AT: 2
    OUT_CHANNELS: 256
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DA_HEADS:
    COS_WEIGHT: 0.1
    DA_IMG_GRL_WEIGHT: 0.01
    DA_INS_GRL_WEIGHT: 0.1
  DEVICE: cuda
  DOMAIN_ADAPTATION_ON: True
  FPN:
    USE_GN: False
    USE_RELU: False
  GROUP_NORM:
    DIM_PER_GP: -1
    EPSILON: 1e-05
    NUM_GROUPS: 32
  KEYPOINT_ON: False
  MASKING_AUGMENTATION: True
  MASKING_BLOC

In [4]:
%cd /content/

/content


In [ ]:
!unzip /content/drive/MyDrive/AI/mic_det_checkpoint.zip -d /content/sample_data/content/MIC/det/configs/da_faster_rcnn

In [7]:
!gdown 1AbsqakY1wtRzGYc6BBZ5W2eN0suLEDkR 

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1AbsqakY1wtRzGYc6BBZ5W2eN0suLEDkR 

